<a href="https://colab.research.google.com/github/KarinaRmzG/TT-II-EVC-CNN/blob/main/recorte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pruebas de casos



*   Este codigo realiza la prediccion de EVC en los casos dados



In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

def preprocess_image(image_path, tamaño_esperado=224):
    img = cv2.imread(image_path)

    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Aplicar umbralización para resaltar los contornos
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

    # Encontrar contornos en la imagen umbralizada
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Seleccionar el contorno más grande basado en el área
    largest_contour = max(contours, key=cv2.contourArea)

    # Calcular puntos extremos (izquierdo, derecho, superior, inferior) del contorno más grande
    leftmost = tuple(largest_contour[largest_contour[:, :, 0].argmin()][0])
    rightmost = tuple(largest_contour[largest_contour[:, :, 0].argmax()][0])
    topmost = tuple(largest_contour[largest_contour[:, :, 1].argmin()][0])
    bottommost = tuple(largest_contour[largest_contour[:, :, 1].argmax()][0])

    # Recortar la región de interés utilizando los puntos extremos
    cropped_img = img[topmost[1]:bottommost[1], leftmost[0]:rightmost[0]]

    # Redimensionar la imagen recortada a 224x224
    cropped_img_resized = cv2.resize(cropped_img, (tamaño_esperado, tamaño_esperado))

    # Normalizar la imagen recortada redimensionada
    cropped_img_resized = cropped_img_resized / 255.0

    return cropped_img_resized

def show_images_with_predictions(model, directorio_imagenes, batch_size=12):
    # lista de nombres de archivos en el directorio
    archivos = os.listdir(directorio_imagenes)

    # Configura la visualización en una cuadrícula de 2x6
    plt.figure(figsize=(20, 8))

    # Determina la longitud del lote
    length = min(batch_size, len(archivos))

    # Itera sobre el lote y muestra las imágenes con sus predicciones
    for i in range(length):
        imagen_nombre = archivos[i]
        imagen_path = os.path.join(directorio_imagenes, imagen_nombre)
        imagen_preprocesada = preprocess_image(imagen_path)
        prediccion = model.predict(np.expand_dims(imagen_preprocesada, axis=0))
        clase_predicha = np.argmax(prediccion, axis=1)[0]

        # Muestra la imagen preprocesada
        plt.subplot(2, 6, i + 1)
        plt.imshow(imagen_preprocesada)

        # Obtiene el nombre de la clase correspondiente a la predicción
        class_name = "EVC" if clase_predicha == 1 else "Normal"
        print(clase_predicha)
        # Agrega el título con el nombre de la clase en azul
        plt.title(class_name, color='blue', fontsize=12)
        plt.axis('off')

    # Muestra la visualización
    plt.show()

# Ruta del modelo h5
model_path = 'CNN_EVC_v2.h5'

# Cargar el modelo
model = load_model(model_path)

# Directorio de imágenes de prueba
directorio_imagenes = 'Caso3_evc'

# Mostrar las imágenes con las predicciones en una cuadrícula de 2x6
show_images_with_predictions(model, directorio_imagenes, batch_size=12)